In [11]:
import dash
from dash import html, dcc, Input, Output, dash_table
import pandas as pd
import plotly.express as px
import requests

# Define Layout of App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

app.layout = html.Div([
    html.H1('COVID-19 Vaccine Coverage by Country', style={'textAlign': 'center'}),
    dcc.Interval(id='interval_db', interval=86400000 * 7, n_intervals=0),
    html.Div(id='dashboard-content', children=[
        html.Div(id='mongo-datatable', className='twelve columns'),
        html.Div(id='bar-graph', className='twelve columns'),
    ], className='row')
])

def fetch_data():
    r = requests.get("https://disease.sh/v3/covid-19/vaccine/coverage/countries?lastdays=1")
    data = r.json()

    # Transform the data into a DataFrame
    df = pd.DataFrame(data)
    
    # Extract the last day of vaccine coverage
    df['latest_coverage'] = df['timeline'].apply(lambda x: list(x.values())[0])
    
    # Sort and take the top 10 countries by vaccine coverage
    df = df.sort_values(by='latest_coverage', ascending=False).head(10)
    
    return df

@app.callback(
    Output('mongo-datatable', 'children'),
    Output('bar-graph', 'children'),
    Input('interval_db', 'n_intervals')
)
def update_dashboard(n_intervals):
    df = fetch_data()
    print("Fetched DataFrame:", df)  # Debug print
    
    table = dash_table.DataTable(
        id='our-table',
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_table={'overflowX': 'auto'},
    )
    
    bar_fig = px.bar(df, x='country', y='latest_coverage', title='Top 10 Countries by Vaccine Coverage')
    
    return table, dcc.Graph(figure=bar_fig)

if __name__ == '__main__':
    app.run_server(debug=False)


Fetched DataFrame:         country                 timeline  latest_coverage
41        China  {'8/15/24': 3491077000}       3491077000
90        India  {'8/15/24': 2206868000}       2206868000
207         USA   {'8/15/24': 676728782}        676728782
27       Brazil   {'8/15/24': 486436436}        486436436
91    Indonesia   {'8/15/24': 448199860}        448199860
99        Japan   {'8/15/24': 383747738}        383747738
15   Bangladesh   {'8/15/24': 361742554}        361742554
147    Pakistan   {'8/15/24': 340974144}        340974144
212     Vietnam   {'8/15/24': 266492149}        266492149
125      Mexico   {'8/15/24': 223158993}        223158993
